In [1]:
import sys
sys.path.append("../../wideboost/")

In [2]:
import time
from wideboost.wrappers import wxgb
import xgboost as xgb

import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval
import os

def hyperopt(
    wide,
    n_iters,
    x_train,
    x_valid,
    x_test,
    y_train,
    y_valid,
    y_test,
    y_cols
):
    os.environ['HYPEROPT_FMIN_SEED'] = '123123'
    best_val = 1000
    num_round = 500
    valid_best = 1000
    test_best = 1000
    best_iter = 0
    best_param = {}
    best_bst = None
    
    maximize = False
    eval_metric = 'many_logloss'
    
    spc = (
        {
            'btype': hp.choice('btype',['I', 'In', 'R', 'Rn']),
            'extra_dims': hp.choice('extra_dims', range(-len(y_cols)+1,6)),
            'beta_eta': hp.loguniform('beta_eta', -7, 0),
            'objective': hp.choice('objective',['manybinary:logistic']),
            'eval_metric':hp.choice('eval_metric',[[eval_metric]]),
            'eta': hp.loguniform('eta', -7, 0),
            'max_depth' : hp.choice('max_depth',range(1,11)),
            'subsample': hp.uniform('subsample', 0.5, 1),
            'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
            'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
            'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
            'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
            'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
            'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
        } if wide else {
            'btype': hp.choice('btype',['I']),
            'extra_dims': hp.choice('extra_dims',[0]),
            'beta_eta': hp.choice('beta_eta', [0]),
            'objective': hp.choice('objective',['manybinary:logistic']),
            'eval_metric':hp.choice('eval_metric',[[eval_metric]]),
            'eta': hp.loguniform('eta', -7, 0),
            'max_depth' : hp.choice('max_depth',range(1,11)),
            'subsample': hp.uniform('subsample', 0.5, 1),
            'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
            'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
            'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
            'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
            'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
            'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
        }
    )
    
    def objective(param):
        nonlocal best_val
        nonlocal valid_best
        nonlocal test_best
        nonlocal best_param
        nonlocal num_round
        nonlocal best_iter
        nonlocal best_bst
        
        watchlist = [
            ((x_train, y_train),'train'),
            ((x_test, y_test), 'test'),
            ((x_valid, y_valid), 'valid')
        ]
        param['output_dim'] = len(y_cols)
        param['nthread'] = 2

        results = {}
        wbst = wxgb.fit(
            x_train,
            y_train,
            param, num_round, watchlist,
            evals_result=results,
            early_stopping_rounds=20,
            maximize=maximize,
            verbose_eval=200
        )
        
        new_loss = (
            min(results['valid'][eval_metric]) if not maximize
            else -max(results['valid'][eval_metric])
        )
        
        if new_loss < best_val:
            print("!!!!!!!!!NEW BEST VALUE!!!!!!")
            
            best_val = new_loss
            
            valid_best = (
                min(results['valid'][eval_metric]) if not maximize
                else max(results['valid'][eval_metric])
            )
            test_best = results['test'][eval_metric][
                np.nanargmin(results['valid'][eval_metric]) if not maximize
                else np.nanargmax(results['valid'][eval_metric])
            ]
            best_iter = wbst.xgbobject.best_iteration
            
            best_param = param.copy()
            best_bst = wbst
        
        return {'loss': new_loss, 'status': STATUS_OK }
    
    best = fmin(objective,
        space=spc,
        algo=tpe.suggest,
        max_evals=n_iters)
    
    return {
        'valid_eval': valid_best,
        'test_eval': test_best,
        'best_iter': best_iter,
        'param': best_param.copy(),
        'bst': best_bst
    }

In [3]:
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=False,
    as_supervised=True,
    with_info=True,
)

for i in ds_train.batch(60000):
    a = i
    
for i in ds_test.batch(60000):
    b = i

In [4]:
xup = a[0].numpy().reshape([-1,28*28])
yup = a[1].numpy()

xdown = b[0].numpy().reshape([-1,28*28])
ydown = b[1].numpy()

def _onehot(Y):
    b = np.zeros((Y.shape[0], Y.max().astype(int)+1))
    b[np.arange(Y.shape[0]),Y.astype(int).flatten()] = 1
    return b
def sort_fix(li):
    li.sort()
    return np.array(li)

def get_data(xtrain, ytrain, seed, sizes):
    np.random.seed(seed)
    train_idx = np.random.choice(range(xtrain.shape[0]), round(xtrain.shape[0] * 0.7), replace=False)
    remaining_idx = sort_fix(list(set(range(xtrain.shape[0])) - set(train_idx)))
    valid_idx = np.random.choice(remaining_idx, round(len(remaining_idx) * 0.5), replace=False)
    remaining_idx = sort_fix(list(set(range(xtrain.shape[0])) - set(train_idx) - set(valid_idx)))
    test_idx = np.random.choice(remaining_idx, len(remaining_idx), replace=False)

    train_idx = np.random.choice(train_idx, sizes[0], replace=False)
    valid_idx = np.random.choice(valid_idx, sizes[1], replace=False)
    test_idx = np.random.choice(test_idx, sizes[2], replace=False)

    x_train = xtrain[train_idx[:,None], features]
    x_valid = xtrain[valid_idx[:,None], features]
    x_test = xtrain[test_idx[:,None], features]           
    
    y_train = _onehot(ytrain[train_idx])
    y_valid = _onehot(ytrain[valid_idx])
    y_test =  _onehot(ytrain[test_idx])

    dtrain = xgb.DMatrix(xtrain[train_idx[:,None], features],label=ytrain[train_idx])
    dvalid = xgb.DMatrix(xtrain[valid_idx[:,None], features],label=ytrain[valid_idx])
    dtest = xgb.DMatrix(xtrain[test_idx[:,None], features],label=ytrain[test_idx])

    return dtrain, dvalid, dtest, x_train, x_valid, x_test, y_train, y_valid, y_test

In [5]:
def get_upstream(seed):
    return get_data(xup, yup, seed, sizes = [3_000, 2_500, 2_000])

def get_downstream(seed):
    return get_data(xdown, ydown, seed, sizes = [5_00, 1_500, 1_000])

In [6]:
xgb.__version__

'1.6.2'

In [7]:
%%time
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

iters = 30
upstream_results = []
downstream_results = []
for est in range(iters):
    t0 = time.time()
    np.random.seed(10000+est)
    features = np.random.choice(range(xup.shape[1]), 200, replace=False)
    downstream_dim = np.random.choice(range(9), 5, replace=False)

    dtrain, dvalid, dtest, x_train, x_valid, x_test, y_train, y_valid, y_test = get_upstream(10000 + est)
    upstream_mask = np.ones([10], dtype=bool)
    upstream_mask[downstream_dim] = False
    y_train = y_train[:, upstream_mask]
    y_valid = y_valid[:, upstream_mask]
    y_test = y_test[:, upstream_mask]
    
    o = hyperopt(
        False,
        15,
        x_train,
        x_valid,
        x_test,
        y_train,
        y_valid,
        y_test,
        y_cols=range(np.sum(upstream_mask))
    )
    upstream_results.append(o)
    
    dtrain, dvalid, dtest, x_train, x_valid, x_test, newy_train, newy_valid, newy_test = get_downstream(10000 + est)
    emb_train = o['bst'].xgbobject.predict(dtrain)
    emb_valid = o['bst'].xgbobject.predict(dvalid)
    emb_test = o['bst'].xgbobject.predict(dtest)

    downstream_mask = np.zeros([10], dtype=bool)
    downstream_mask[downstream_dim] = True
    newy_train = newy_train[:, downstream_mask]
    newy_valid = newy_valid[:, downstream_mask]
    newy_test =  newy_test[:, downstream_mask]
    
    d = hyperopt(
        False,
        15,
        emb_train.reshape([x_train.shape[0], -1]),
        emb_valid.reshape([x_valid.shape[0], -1]),
        emb_test.reshape([x_test.shape[0], -1]),
        newy_train,
        newy_valid,
        newy_test,
        y_cols=range(np.sum(downstream_mask))
    )
    t1 = time.time()
    d['time'] = t1 - t0
    downstream_results.append(d)


[0]	train-many_logloss:0.69049	test-many_logloss:0.69060	valid-many_logloss:0.69060
[200]	train-many_logloss:0.36077	test-many_logloss:0.37502	valid-many_logloss:0.37590
[400]	train-many_logloss:0.21686	test-many_logloss:0.24012	valid-many_logloss:0.24236
[499]	train-many_logloss:0.17398	test-many_logloss:0.20080	valid-many_logloss:0.20375
!!!!!!!!!NEW BEST VALUE!!!!!!                         
[0]	train-many_logloss:0.69203	test-many_logloss:0.69203	valid-many_logloss:0.69203
[200]	train-many_logloss:0.52364	test-many_logloss:0.52437	valid-many_logloss:0.52404
[400]	train-many_logloss:0.42603	test-many_logloss:0.42729	valid-many_logloss:0.42690
[499]	train-many_logloss:0.39274	test-many_logloss:0.39422	valid-many_logloss:0.39393
[0]	train-many_logloss:0.69249	test-many_logloss:0.69249	valid-many_logloss:0.69250
[200]	train-many_logloss:0.58147	test-many_logloss:0.58206	valid-many_logloss:0.58219
[400]	train-many_logloss:0.50012	test-many_logloss:0.50123	valid-many_logloss:0.50153
[499]

../../wideboost/wideboost/evals/classification.py:82: RuntimeWarning: invalid value encountered in log
  return -np.mean(np.log(p))

../../wideboost/wideboost/evals/classification.py:82: RuntimeWarning: divide by zero encountered in log
  return -np.mean(np.log(p))

../../wideboost/wideboost/evals/classification.py:80: RuntimeWarning: overflow encountered in exp
  p = 1/(1 + np.exp(-logits))

../../wideboost/wideboost/objectives/manybinarylogloss.py:14: RuntimeWarning: overflow encountered in exp
  P = 1 / (1 + np.exp(-logits))



[20]	train-many_logloss:nan	test-many_logloss:nan	valid-many_logloss:nan
[0]	train-many_logloss:0.67305	test-many_logloss:0.67383	valid-many_logloss:0.67393
[200]	train-many_logloss:0.03589	test-many_logloss:0.08421	valid-many_logloss:0.09333
[400]	train-many_logloss:0.00987	test-many_logloss:0.06856	valid-many_logloss:0.07941
[487]	train-many_logloss:0.00655	test-many_logloss:0.06756	valid-many_logloss:0.07849
!!!!!!!!!NEW BEST VALUE!!!!!!                                         
[0]	train-many_logloss:0.51780	test-many_logloss:0.51969	valid-many_logloss:0.52098
[79]	train-many_logloss:0.00988	test-many_logloss:0.08245	valid-many_logloss:0.09396
[0]	train-many_logloss:0.69094	test-many_logloss:0.69104	valid-many_logloss:0.69103
[200]	train-many_logloss:0.39574	test-many_logloss:0.40909	valid-many_logloss:0.40977
[400]	train-many_logloss:0.25212	test-many_logloss:0.27410	valid-many_logloss:0.27594
[499]	train-many_logloss:0.20689	test-many_logloss:0.23235	valid-many_logloss:0.23468
[0]

[499]	train-many_logloss:0.46302	test-many_logloss:0.46364	valid-many_logloss:0.46369
[0]	train-many_logloss:0.16189	test-many_logloss:0.23760	valid-many_logloss:0.23462
[26]	train-many_logloss:0.00077	test-many_logloss:0.10683	valid-many_logloss:0.10047
!!!!!!!!!NEW BEST VALUE!!!!!!                                         
[0]	train-many_logloss:0.68975	test-many_logloss:0.68975	valid-many_logloss:0.68974
[200]	train-many_logloss:0.36165	test-many_logloss:0.36145	valid-many_logloss:0.36109
[400]	train-many_logloss:0.27316	test-many_logloss:0.27424	valid-many_logloss:0.27378
[499]	train-many_logloss:0.25093	test-many_logloss:0.25315	valid-many_logloss:0.25247
[0]	train-many_logloss:0.68798	test-many_logloss:0.68810	valid-many_logloss:0.68805
[200]	train-many_logloss:0.23009	test-many_logloss:0.24736	valid-many_logloss:0.24398
[400]	train-many_logloss:0.11457	test-many_logloss:0.14334	valid-many_logloss:0.13879
[499]	train-many_logloss:0.08752	test-many_logloss:0.12070	valid-many_loglos

100%|██████████| 15/15 [00:44<00:00,  2.95s/trial, best loss: 0.19681]
[0]	train-many_logloss:0.69052	test-many_logloss:0.69064	valid-many_logloss:0.69060
[200]	train-many_logloss:0.36856	test-many_logloss:0.38329	valid-many_logloss:0.37988
[400]	train-many_logloss:0.22699	test-many_logloss:0.25193	valid-many_logloss:0.24706
[499]	train-many_logloss:0.18435	test-many_logloss:0.21357	valid-many_logloss:0.20842
!!!!!!!!!NEW BEST VALUE!!!!!!                         
[0]	train-many_logloss:0.69203	test-many_logloss:0.69203	valid-many_logloss:0.69202
[200]	train-many_logloss:0.52575	test-many_logloss:0.52622	valid-many_logloss:0.52380
[400]	train-many_logloss:0.42959	test-many_logloss:0.43051	valid-many_logloss:0.42626
[499]	train-many_logloss:0.39684	test-many_logloss:0.39799	valid-many_logloss:0.39299
[0]	train-many_logloss:0.69250	test-many_logloss:0.69250	valid-many_logloss:0.69249
[200]	train-many_logloss:0.58262	test-many_logloss:0.58338	valid-many_logloss:0.58189
[400]	train-many_log

...

[400]	train-many_logloss:0.49776	test-many_logloss:0.49886	valid-many_logloss:0.49996
[499]	train-many_logloss:0.46489	test-many_logloss:0.46625	valid-many_logloss:0.46755
[0]	train-many_logloss:0.16504	test-many_logloss:0.24407	valid-many_logloss:0.24767
[45]	train-many_logloss:0.00055	test-many_logloss:0.10929	valid-many_logloss:0.10534
!!!!!!!!!NEW BEST VALUE!!!!!!                                         
[0]	train-many_logloss:0.68972	test-many_logloss:0.68973	valid-many_logloss:0.68974
[200]	train-many_logloss:0.35833	test-many_logloss:0.35989	valid-many_logloss:0.36151
[400]	train-many_logloss:0.27089	test-many_logloss:0.27411	valid-many_logloss:0.27623
[499]	train-many_logloss:0.24965	test-many_logloss:0.25369	valid-many_logloss:0.25588
[0]	train-many_logloss:0.68801	test-many_logloss:0.68805	valid-many_logloss:0.68810
[200]	train-many_logloss:0.23336	test-many_logloss:0.24685	valid-many_logloss:0.24921
[400]	train-many_logloss:0.11745	test-many_logloss:0.14179	valid-many_loglos

[499]	train-many_logloss:0.43533	test-many_logloss:0.47119	valid-many_logloss:0.46834
100%|██████████| 15/15 [00:28<00:00,  1.89s/trial, best loss: 0.215066]
[0]	train-many_logloss:0.69050	test-many_logloss:0.69059	valid-many_logloss:0.69058
[200]	train-many_logloss:0.35762	test-many_logloss:0.37084	valid-many_logloss:0.37261
[400]	train-many_logloss:0.21252	test-many_logloss:0.23460	valid-many_logloss:0.23690
[499]	train-many_logloss:0.16966	test-many_logloss:0.19544	valid-many_logloss:0.19778
!!!!!!!!!NEW BEST VALUE!!!!!!                         
[0]	train-many_logloss:0.69204	test-many_logloss:0.69204	valid-many_logloss:0.69204
[200]	train-many_logloss:0.52512	test-many_logloss:0.52538	valid-many_logloss:0.52610
[400]	train-many_logloss:0.42766	test-many_logloss:0.42811	valid-many_logloss:0.42944
[499]	train-many_logloss:0.39405	test-many_logloss:0.39456	valid-many_logloss:0.39616
[0]	train-many_logloss:0.69247	test-many_logloss:0.69247	valid-many_logloss:0.69247
[200]	train-many_lo

[0]	train-many_logloss:0.69202	test-many_logloss:0.69211	valid-many_logloss:0.69212
[200]	train-many_logloss:0.51885	test-many_logloss:0.53302	valid-many_logloss:0.53436
[400]	train-many_logloss:0.40871	test-many_logloss:0.43501	valid-many_logloss:0.43749
[499]	train-many_logloss:0.36817	test-many_logloss:0.40009	valid-many_logloss:0.40288
[0]	train-many_logloss:0.68959	test-many_logloss:0.68957	valid-many_logloss:0.68961
[200]	train-many_logloss:0.36807	test-many_logloss:0.36625	valid-many_logloss:0.37105
[400]	train-many_logloss:0.29961	test-many_logloss:0.29989	valid-many_logloss:0.30660
[499]	train-many_logloss:0.28458	test-many_logloss:0.28671	valid-many_logloss:0.29387
[0]	train-many_logloss:0.56085	test-many_logloss:0.56295	valid-many_logloss:0.56459
[62]	train-many_logloss:0.10701	test-many_logloss:0.22000	valid-many_logloss:0.23288
[0]	train-many_logloss:0.69242	test-many_logloss:0.69249	valid-many_logloss:0.69250
[200]	train-many_logloss:0.56617	test-many_logloss:0.58008	vali

[0]	train-many_logloss:0.24104	test-many_logloss:0.37603	valid-many_logloss:0.38446
[20]	train-many_logloss:nan	test-many_logloss:nan	valid-many_logloss:nan
[0]	train-many_logloss:0.67500	test-many_logloss:0.67654	valid-many_logloss:0.67632
[194]	train-many_logloss:0.08941	test-many_logloss:0.24000	valid-many_logloss:0.22777
!!!!!!!!!NEW BEST VALUE!!!!!!                                         
[0]	train-many_logloss:0.53628	test-many_logloss:0.54598	valid-many_logloss:0.54445
[42]	train-many_logloss:0.06624	test-many_logloss:0.27885	valid-many_logloss:0.26270
[0]	train-many_logloss:0.69121	test-many_logloss:0.69140	valid-many_logloss:0.69137
[200]	train-many_logloss:0.43360	test-many_logloss:0.46198	valid-many_logloss:0.45995
[400]	train-many_logloss:0.30701	test-many_logloss:0.36047	valid-many_logloss:0.35661
[499]	train-many_logloss:0.26636	test-many_logloss:0.33124	valid-many_logloss:0.32676
[0]	train-many_logloss:0.60362	test-many_logloss:0.60533	valid-many_logloss:0.60532
[133]	t

[499]	train-many_logloss:0.49836	test-many_logloss:0.49979	valid-many_logloss:0.50059
[0]	train-many_logloss:0.22521	test-many_logloss:0.34659	valid-many_logloss:0.35121
[22]	train-many_logloss:0.00678	test-many_logloss:0.34463	valid-many_logloss:0.37946
[0]	train-many_logloss:0.68989	test-many_logloss:0.68995	valid-many_logloss:0.68995
[200]	train-many_logloss:0.37469	test-many_logloss:0.38089	valid-many_logloss:0.38211
[400]	train-many_logloss:0.29604	test-many_logloss:0.30612	valid-many_logloss:0.30920
[499]	train-many_logloss:0.27885	test-many_logloss:0.29091	valid-many_logloss:0.29482
!!!!!!!!!NEW BEST VALUE!!!!!!                                         
[0]	train-many_logloss:0.68846	test-many_logloss:0.68868	valid-many_logloss:0.68871
[200]	train-many_logloss:0.27305	test-many_logloss:0.31506	valid-many_logloss:0.31809
[400]	train-many_logloss:0.16400	test-many_logloss:0.23595	valid-many_logloss:0.24275
[499]	train-many_logloss:0.13609	test-many_logloss:0.22111	valid-many_loglos

In [8]:
import pandas as pd
pd.DataFrame(downstream_results)

,valid_eval,test_eval,best_iter,param,bst,time
0,0.190711,0.211871,207,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7f96...,433.809744
1,0.196810,0.213637,215,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7f96...,471.955726
2,0.225822,0.212136,179,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7f96...,421.917357
3,0.233279,0.229543,178,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7f96...,395.622992
4,0.198027,0.198238,192,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7f96...,398.652405
5,0.236596,0.238406,161,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7f96...,392.882304
6,0.198536,0.200164,179,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7f96...,403.291636
7,0.248428,0.230623,166,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7f96...,406.822110
8,0.194050,0.204273,177,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7f96...,402.755913
9,0.227076,0.235251,197,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7f96...,402.669511
